## Rhythmic/Periodic Move Sequences

___


This notebook highlights my progress thus far explorating a solution strategy based qualitatively more on cube movement than on individual cube states. Taking advantage of the its periodic nature, I developed a [CyclicSolver](../rubiks/solver/CyclicSolver.py) class designed to formalize the Rubik's cube's foundational periodic movement cycles. The periodic cycles of any **single side** are summarized in the table below. From these, we can build up a set of **two-sided** periodic move cycles, as further illustrated in the following cells:


| Cycle Period  | Single Side Rotations (degrees) |
| :------------ | :----------- |
| Two Moves:    | **(90,-90), (-90,90), (180,180)** |
| Three Moves:  | **(180,90,90), (90,180,90), (90,90,180)** |
| Also 3 Moves: | **(180,-90,-90), (-90,180,-90), (-90,-90,180)** |
| Four Moves:   | **(90,90,90,90), (-90,-90,-90,-90)** |

___


In [ ]:
import sys

# This for managing relative imports from nb
if '..' not in sys.path: sys.path.append('..')

import numpy as np
import matplotlib
import matplotlib.pyplot as plt

from rubiks.model.CubeView import CubeView
from rubiks.model.DirectCube import DirectCube
from rubiks.model.VectorCube import VectorCube, color_name, color_letr

from rubiks.solver.CyclicSolver import CyclicSolver
from rubiks.solver.DirectSolver import DirectSolver

In [ ]:
# This generates the cycles
csolver = CyclicSolver()

In [ ]:
matplotlib.rcParams['figure.figsize'] = (20, 4)

print(f"Total number of 2-sided move cycles: {len(csolver.periods)}",
      f"\nUnique cycle periods: {sorted(set(csolver.periods))}",
      f"\nAverage cycle period: {np.average(csolver.periods)}\n")

plt.figure(1)
plt.plot(csolver.periods)
plt.title(f'Rubik Cycle Periods')
plt.ylabel('Number of Rotations')
plt.xlabel('Cycle Bin')

plt.figure(2)
plt.plot(csolver.periods[:int(len(csolver.cycles) / 12)])
plt.title(f'Rubik Cycle Periods (Zoomed In)')
plt.ylabel('Number of Rotations')
plt.xlabel('Cycle Bin')

print("Example selection of two-sided cycles and periods:")
for eg in np.random.randint(0, len(csolver.cycles), size=10): 
    print(f"  Cycle {eg+1} (period: {csolver.periods[eg]})",
          [f"{color_letr(mv[0])}:({mv[1]})" for mv in csolver.cycles[eg]])
print("\n")

## Measuring Cube Entropy/Order/Disorder

___

**Facelet Dimensions (in integer**  
**units as defined in VectorCube):**

![alt_text](images/rubiks_dimensions.png)

My initial attempt to measure cube entropy used a squared difference vector between adjacent facelets. As discernible from the VectorCube dimensions above, in a solved or "ordered" orientation this squared difference vector will be (4,0,0), (0,4,0), or (0,0,4). The basic "Order" heuristic sums over all occurences of these vectors for all approriate adjacent facelet pairs . Note that it does NOT take into account the correct "facing" orientation between adjacent facelets, that requires the more complicated "Dance" heuristic measurement. The following cells demonstrate the difference between these two measurements using an example facelet pair.

___


In [ ]:
# Demonstrates (using random sampling) a set of cube permutations in which
# a specific pair of adjacent corner and edge facelets are "ordered" and
# additionally, which of those are also oriented "face-to-face"

ordered = {}
I4 = np.array([[4,0,0],[0,4,0],[0,0,4]])

for i in range(10000):
#{
    cube = DirectCube().scramble()
    diff_sq = (cube.facelet_matrix[2:,18] - cube.facelet_matrix[2:,19])**2
    if np.sum(np.sum(I4 == np.broadcast_to(diff_sq, (3,3)), axis=1) == 3) > 0:
    #{
        lead_dir = cube.direction_matrix[2:, DirectCube.get_direction_index([19])[0]]
        foll_dir = cube.direction_matrix[2:, DirectCube.get_direction_index([18])[0]]
        k_pos = tuple(cube.facelet_matrix[2:,[18,19]].T.flatten())
        k_sort = np.nonzero(cube.facelet_matrix[2:,18]**2 == 9)[0][0]
        if sum(cube.facelet_matrix[2:,18] == -3) > 0: k_sort += 3
        
        if np.sum(lead_dir + foll_dir) == 0:
            ordered[(k_sort, k_pos, 'f')] = cube.state()
        else: ordered[(k_sort, k_pos, 'o')] = cube.state()
    #}
#}

flet_cn = DirectCube().facelet_matrix[:,18]
flet_ed = DirectCube().facelet_matrix[:,19]
print(f"Permutations of Facelet Pair:\n"
      f"  {color_name(flet_cn[0])}[{flet_cn[1]}] of (home/solved) position {flet_cn[2:]}\n"
      f"  {color_name(flet_ed[0])}[{flet_ed[1]}] of (home/solved) position {flet_ed[2:]}\n")

last_ksort = None
view = CubeView(DirectCube())
idx = np.concatenate((VectorCube._centers, [18,19]))
for i, k in enumerate(sorted(ordered.keys())):
    if last_ksort is None: last_ksort = k[0]        
    elif last_ksort != k[0]:
        print("---------------------------")
        last_ksort = k[0]
        view.draw_snapshots()
        view.reset_snapshots()
    
    view.viewable_cube.reset(state=ordered[k])
    if k[2] == 'o': view.push_snapshot(flet_idx=idx, caption=f"#{i+1}")
    elif view.viewable_cube.solved(flet_index=[18,19]):
        view.push_snapshot(flet_idx=idx, caption=f"#{i+1}:  Face-to-Face (solved)")
    else: view.push_snapshot(flet_idx=idx, caption=f"#{i+1}:  Face-to-Face")

print("---------------------------")
view.draw_snapshots()

## Minimizing Entropy Across Periodic Cycles

___


The following cells demonstrate the surprising result of a relatively unsophisticated search for maximally ordered cube permutations across the 2-sided cyclic move sequences. Though the solutions are far from optimal with respect to number of moves (i.e. far greater than God's number of 20 moves to a solution), they often complete the F2L step and at times even the OLL and definitely perk my interest for further exploration here.

For a given cube scramble below, several searches are conducted using various move-depths (meaning the number of moves down each cycle a search is allowed to proceed). Any set of integer search depths may be chosen, as well as the special values: 'length' (indicating only the base length of each cycle is used, i.e. [4, 5, 6, 7, or 8]), or 'period' (indicated the full period of each cycle is used, i.e. [12, 15, 24, 30, 60, 75, 90, 105, 126, 180, 210, 270, or 315]). Of course, greater depths require greater execution times; with the default search depths below, my machine requires 45-50 minutes to complete the following cells (10-15 then 30-35).

Interestingly, the less precise 'order' heuristic seems to produce comparable or better results than the 'dance' heuristic. And though not always the case, a depth of 30 or 60 seems a kind of sweet spot, often coming closest to a full solution (defined by an order heuristic value of 72, or a dance heuristic value of 216, which incidentally, is measuring a few additional quantities beyond just the "face-to-face" orientation). For a detailed understanding of the heuristic, please see the dance_heuristics function in [DirectSolver](../rubiks/solver/DirectSolver.py).
  
___



In [ ]:
%%time

# 'Order' Heuristic Search

i_times = []
starting_cube = DirectCube().scramble(sz=64)
view = CubeView(DirectCube(starting_cube)).push_snapshot()

# I found the smaller set of search_depths below generally sufficient to demonstrate the
# potential of this search; feel free to uncomment the larger set or explore your own:
# ---------------------------------------------------------------------------------------
# search_depths = ['length', 12, 15, 24, 30, 60, 75, 90, 105, 126, 180, 210, 270, 315, 'period']
search_depths = ['length', 24, 30, 60, 'period']

for nmoves in search_depths:
#{
    cube, moves, i_time = csolver.cycle_solve(starting_cube, DirectSolver.order_heuristic, nmoves=nmoves, verbose=False)

    i_times.append(i_time)
    value = DirectSolver.order_heuristic(cube)
    view.viewable_cube.reset(state=cube.state())
    view.push_snapshot(caption=f"Cycle depth: {nmoves} | Moves: {len(moves)} | Val: {value}")
    print(f"Cycle depth: {nmoves} | Moves: {len(moves)} | Order Value: {value} | Search Time: {int(sum(i_time))} sec")
#}

plt.figure(1)
for t in i_times: plt.plot(t)
plt.title(f"Time Per Iteration (using 'Order' Heuristic)")
plt.ylabel('Seconds')
plt.xlabel('Iterations')
plt.legend(search_depths, loc='upper right')
view.draw_snapshots()

In [ ]:
%%time

# 'Dance' Heuristic Search 
# (using same starting_cube and search_depths as above)

i_times = []
view = CubeView(DirectCube(starting_cube)).push_snapshot()

for nmoves in search_depths:
#{
    cube, moves, i_time = csolver.cycle_solve(starting_cube, DirectSolver.dance_heuristics, nmoves=nmoves, verbose=False)

    i_times.append(i_time)
    value = DirectSolver.dance_heuristics(cube)
    view.viewable_cube.reset(state=cube.state())
    view.push_snapshot(caption=f"Cycle depth: {nmoves} | Moves: {len(moves)} | Val: {value}")
    print(f"Cycle depth: {nmoves} | Moves: {len(moves)} | Dance Value: {value} | Search Time: {int(sum(i_time))} sec")
#}

plt.figure(2)
for t in i_times: plt.plot(t)
plt.title(f"Time Per Iteration (using 'Dance' Heuristic)")
plt.ylabel('Seconds')
plt.xlabel('Iterations')
plt.legend(search_depths, loc='upper right')
view.draw_snapshots()